##### Imports

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List


In [11]:
dataset = pd.read_csv("./datasets/mat.csv")
dataset.columns=[i for i in range(dataset.shape[1])]
dataset

,0,1,2,3,4
0,4,5,3,6,1
1,8,10,6,12,2
2,3,3,3,3,3
3,7,1,9,0,8
4,14,2,18,0,16


In [12]:
dataset.shape[0]

5

In [13]:
# generate a random matrix
lines = dataset.shape[0]
columns = dataset.shape[1]

matrix_2 = [[]] * lines
for i in range(len(dataset) - 1):
    for j in range(i + 1, len(dataset)):
        diff = np.diff((dataset[i], dataset[j]), axis=0) * (-1)
        matrix_2 = np.concatenate((matrix_2, diff.T), axis=1)

print(dataset)
print("-----------------------------------")
print(matrix_2)
print("-----------------------------------")

    0   1   2   3   4
0   4   5   3   6   1
1   8  10   6  12   2
2   3   3   3   3   3
3   7   1   9   0   8
4  14   2  18   0  16
-----------------------------------
[[ -1.   1.  -2.   3.   2.  -1.   4.  -3.   2.   5.]
 [ -2.   2.  -4.   6.   4.  -2.   8.  -6.   4.  10.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  6.  -2.   7.  -1.  -8.   1.  -7.   9.   1.  -8.]
 [ 12.  -4.  14.  -2. -16.   2. -14.  18.   2. -16.]]
-----------------------------------


In [14]:
matrix_3 = matrix_2 / (np.max(matrix_2) - np.min(matrix_2))
print(matrix_3)

[[-0.02941176  0.02941176 -0.05882353  0.08823529  0.05882353 -0.02941176
   0.11764706 -0.08823529  0.05882353  0.14705882]
 [-0.05882353  0.05882353 -0.11764706  0.17647059  0.11764706 -0.05882353
   0.23529412 -0.17647059  0.11764706  0.29411765]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.17647059 -0.05882353  0.20588235 -0.02941176 -0.23529412  0.02941176
  -0.20588235  0.26470588  0.02941176 -0.23529412]
 [ 0.35294118 -0.11764706  0.41176471 -0.05882353 -0.47058824  0.05882353
  -0.41176471  0.52941176  0.05882353 -0.47058824]]


In [15]:
histogram = [0] * 21


In [44]:
# create empty histogram with 21 (10<0 +0+ 10 >0) with zeros
histogram = [0] * 21
# helper variable
j = -1
# now create interval values of histogram to use them later for discretization
# [-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(21):
    histogram[i] = j
    j = round(j + 0.1, 1)
# now we calculate the histogram
# we start by [-1,0.9[
for i in range(len(histogram)):
    val = histogram[i]
    histogram[i] = 0
    for j in range(len(matrix_3)):
        for k in range(len(matrix_3[0])):
            if matrix_3[j][k] >= val and matrix_3[j][k] < round(val + 0.1, 1):
                histogram[i] += 1
# now we have a regular histogram
# we need to normalize it
histogram = np.array(histogram)
print(histogram / sum(histogram))

[0.   0.   0.   0.   0.   0.06 0.   0.06 0.06 0.18 0.38 0.12 0.08 0.02
 0.02 0.02 0.   0.   0.   0.   0.  ]


In [43]:
def OTSU(histogram: List[float]) -> float:
    total = sum(histogram)
    helper = [0] * 21
    j = -1
    max_seil_b = -float("inf")
    result = -1
    result1 = -1
    for i in range(21):
        helper[i] = j
        j = round(j + 0.1, 1)
   #first part -1..0 seuil negative
    for i in range((len(histogram)//2)  ):
        sum1 = 0.00001
        sum2 = 0.00001
        xfb = 0
        xbg = 0
        for j in range(0, i):
            xbg += helper[j] * histogram[j]
            sum1 += histogram[j]
        for j in range(i, len(histogram)):
            xfb += helper[j] * histogram[j]
            sum2 += histogram[j]
        xbg /= sum1
        xfb /= sum2
        wb = sum1 / total
        wf = sum2 / total
        seil_b = wb * wf * ((xbg - xfb) * (xbg - xfb))
        if seil_b > max_seil_b:
            max_seil_b = seil_b
            result = helper[i]
        #seconde part 0..1 seuil positive
        for k in range(len(histogram)//2 ,len(histogram)   ):
            sum1 = 0.00001
            sum2 = 0.00001
            xfb = 0
            xbg = 0
            
            for j in range(0, k):
                xbg += helper[j] * histogram[j]
                sum1 += histogram[j]
            
            for j in range(k, len(histogram)):
                xfb += helper[j] * histogram[j]
                sum2 += histogram[j]
            xbg /= sum1
            xfb /= sum2
            wb = sum1 / total
            wf = sum2 / total
            seil_b = wb * wf * ((xbg - xfb) * (xbg - xfb))
            if seil_b > max_seil_b:
                max_seil_b = seil_b 
                result1 = helper[k]   
    return result , result1


print("histogramm ",histogram)
print(OTSU(histogram))

histogramm  [ 0  0  0  0  0  3  0  3  3  9 19  6  4  1  1  1  0  0  0  0  0]
(-0.1, 0.0)
